## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
import json
sys.path.append('..')
import stober_synthesis_utils as stober
import usaxs_utils

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = 'APSsetup.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
response in connect:  [True, True, True, True]


In [5]:
safe_pos = (5, 200, 100)
cell_pos = (217.5, 69.0, 61.5)
url = 'http://192.168.1.4'
port = '5000'
name = 'PSL'
username = 'test'
password = 'domo_arigato'

In [6]:
psl = PneumaticSampleLoader.PneumaticSampleLoader(url, port, name, cell_pos, safe_pos, username, password)

status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': True, \'piston-vent\': False, \'postsample\': True}","Rinse Program Step 3/6: blow for 15s","[250318-03:11:03] Load stopped at voltage mean = 3.7568305206298827 and stdev = 1.1343200899231254"]\n'


In [7]:
jubilee.load_tool(psl)

In [9]:
sample_table = pd.read_csv('APS_InitialSetupSamples - Sheet1.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [10]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [11]:
sample_table['well'] = ['A1', 'A2', 'A3', 'A4']

In [12]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_volume,well
0,silica_free_blank,1,0,733,1504,2746,2190,2700,A1
1,control_1,2,131,733,1504,2746,2190,2700,A2
2,control_2,3,131,733,1504,2746,2190,2700,A3
3,control_3,4,131,733,1504,2746,2190,2700,A4


In [13]:
sample_table['water_volume'].sum()

6016

## Load Tools

In [15]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}
Syringe name:  1cc_4
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_4'}


In [16]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

In [17]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [18]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [19]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [20]:
water_syringe.load_syringe(0, water_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load ammonia syringe

In [21]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [22]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [89]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [91]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [92]:
ammonia_syringe.load_syringe(850, ammonia_syringe.full_position-1)

Loaded syringe, remaining volume 850 uL


## Load TEOS syringe

In [26]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [27]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [28]:
teos_syringe.load_syringe(0, teos_syringe.empty_position - 1)

Loaded syringe, remaining volume 0 uL


## load ethanol syringe

In [29]:
#jubilee.park_tool()

In [30]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [31]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


## Don't need to load surfactant syringe

In [32]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load AFL

## Load labware


In [34]:
samples_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples_1.manual_offset([(30.3, 168.4), (120.1, 169.4), (121.6, 112.4)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [35]:
samples_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 0)
samples_2.manual_offset([(31.4, 72.4), (121.1, 71.9), (122.9, 16.9)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [37]:
stocks_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_1.manual_offset([(29.1, 265.0), (119.8, 265.0), (120.8, 209.6)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


stocks_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_2.manual_offset([(169.8, 265.2), (260.3, 265.2), (260.3, 210.2)])

In [45]:
teos_stocks = stober.Stocks([stocks_1[0]], usable_volume= 17000)
ammonia_stocks = stober.Stocks([stocks_1[1], stocks_1[2]], usable_volume= 17000)
water_stocks = stober.Stocks([stocks_1[3], stocks_1[4], stocks_1[5], stocks_1[6]], usable_volume= 17000)
ethanol_stocks = stober.Stocks([stocks_1[7], stocks_1[8], stocks_1[9], stocks_1[10], stocks_1[11]], usable_volume = 17000)
f127_stocks = stober.Stocks([stocks_2[0], stocks_2[1], stocks_2[2]], usable_volume = 17000)
ctab_stocks = stober.Stocks([stocks_2[3], stocks_2[4]], usable_volume= 17000)

rinse_stocks_precursor = [stocks_2[5], stocks_2[6], stocks_2[7]]
rinse_stocks_teos = [stocks_2[8], stocks_2[9], stocks_2[10]]

## Check sample alignment

In [47]:
jubilee.pickup_tool(mix_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

169.0


Is the tip centered over sample A1? 


169.0


Is the tip centered over sample A2? 


169.0


...

Is the tip centered over sample B4? 


113.0


Is the tip centered over sample C1? 


113.0


Is the tip centered over sample C2? 


In [49]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B2
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B3
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B4
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C1
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C2


In [57]:
jubilee.park_tool()


In [106]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

175.8614659806774


# Experiment


### Load ethanol background

In [59]:
psl.prepare_cell()

status r code 200
status:  b'["State: LOADED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","[250318-10:27:40] Load stopped at voltage mean = 4.5493027877807615 and stdev = 0.8819717019590799"]\n'
status r code 200
status:  b'["State: LOADED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","[250318-10:27:40] Load stopped at voltage mean = 4.5493027877807615 and stdev = 0.8819717019590799"]\n'
status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'r

In [60]:
psl.load_sample(mix_syringe, ethanol_stocks.get_available_stock(2000), 2000)

status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","[250318-10:27:40] Load stopped at voltage mean = 4.5493027877807615 and stdev = 0.8819717019590799"]\n'
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","[250318-10:27:40] Load stopped at voltage mean = 4.5493027877807615 and stdev = 0.8819717019590799"]\n'
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': Fal

In [51]:
ethanol_stocks.get_available_stock(500)

Well B4 form 20mLscintillation 12 Well Plate 18000 µL on slot 4

In [61]:
ethanol_stocks.stock_list_lookup

{'B4': Well B4 form 20mLscintillation 12 Well Plate 18000 µL on slot 4,
 'C1': Well C1 form 20mLscintillation 12 Well Plate 18000 µL on slot 4,
 'C2': Well C2 form 20mLscintillation 12 Well Plate 18000 µL on slot 4,
 'C3': Well C3 form 20mLscintillation 12 Well Plate 18000 µL on slot 4,
 'C4': Well C4 form 20mLscintillation 12 Well Plate 18000 µL on slot 4}

In [62]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_volume,well
0,silica_free_blank,1,0,733,1504,2746,2190,2700,A1
1,control_1,2,131,733,1504,2746,2190,2700,A2
2,control_2,3,131,733,1504,2746,2190,2700,A3
3,control_3,4,131,733,1504,2746,2190,2700,A4


In [63]:
location_lookup = {row['uuid']:samples_1[row['well']] for i, row in sample_table.iterrows()}

In [64]:
sample_table['ethanol_volume'].sum()

10984

## add ethanol

In [99]:
ind = 3

well = location_lookup[ind+1]
etoh_vol = sample_table.iloc[ind-1]['ethanol_volume']
f127_volume = sample_table.iloc[ind-1]['f127_volume']
ctab_volume = sample_table.iloc[ind-1]['ctab_volume']
teos_volume = sample_table.iloc[ind-1]['teos_volume']
ammonia_volume = sample_table.iloc[ind-1]['ammonia_volume']
water_volume = sample_table.iloc[ind-1]['water_volume']
sample_table_select = sample_table.iloc[ind:ind+1]
location_lookup_selected = {ind+1:well}

sample_uid = sample_table.iloc[ind-1]['uuid']
sample_name = f'Control_{ind}'

#composition = json.dumps({'teos_vol':teos_volume, 'ammonia_vol':ammonia_volume, 'water_volume':water_volume, 'ethanol_volume':etoh_vol, 'ctab_volume':ctab_volume, 'f127_volume':f127_volume})

reaction_time = 60*5

In [100]:
sample_table_select

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_volume,well
3,control_3,4,131,733,1504,2746,2190,2700,A4


In [101]:
location_lookup_selected

{4: Well A4 form 20mLscintillation 12 Well Plate 18000 µL on slot 2}

In [102]:
well

Well A4 form 20mLscintillation 12 Well Plate 18000 µL on slot 2

In [103]:
ethanol_stocks.get_current_volumes()

{'B4': 3008.0, 'C1': 13754.0, 'C2': 17000, 'C3': 17000, 'C4': 17000}

In [111]:
psl.prepare_cell()

status r code 200
status:  b'["State: LOADED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","[250318-13:41:25] Load stopped at voltage mean = 4.11986141204834 and stdev = 1.093001251841383"]\n'
status r code 200
status:  b'["State: LOADED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","[250318-13:41:25] Load stopped at voltage mean = 4.11986141204834 and stdev = 1.093001251841383"]\n'
status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\

In [75]:
jubilee.park_tool()

In [105]:

start_time = time.time()

#add ethanol
stober.reactant_transfer(jubilee, mix_syringe, ethanol_stocks, well, etoh_vol, 500)

#add water
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup, 'water_volume', water_stocks, dwell_time = 7)
print(f'added water')
#add ammonia
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup, 'ammonia_volume', ammonia_stocks, dwell_time = 10, refill_dwell=5)
print('added ammonia')
# add F127

stober.reactant_transfer(jubilee, surfactant_syringe, f127_stocks, well, f127_volume, 50, rinse_stocks=rinse_stocks_precursor)
print(f'added {f127_volume} f127')

stober.reactant_transfer(jubilee, surfactant_syringe, ctab_stocks, well, ctab_volume, 50, rinse_stocks=rinse_stocks_precursor)
print(f'added {ctab_volume} uL CTAB')
# mix precursors

stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, rinse_stocks_precursor, 3, n_rinse = 1)
print('mixed samples')
# add teos
teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)

logger.info(f'Added TEOS to sample {ind} at {time.strftime("%H:%M", time.localtime(teos_add_time))}')

print(f'Time from start to TEOS addition: {teos_add_time - start_time} s')

time.sleep(reaction_time - (time.time() - teos_add_time))

print('########## Complete ############')

psl.load_sample(mix_syringe, well, 1500)
usaxs_utils.trigger_usaxs_measurement(sample_name, sample_uid, 'ControlComposition')

mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

print(f'Time to load: {(time.time() - start_time)/60}')
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock, s_aspirate = 2000, s_dispense = 1000)

jubilee.park_tool()

end_time = time.time()

print(f'Total sample time: {(end_time - start_time)/60}')




Loaded syringe, remaining volume 0 uL
breaking dispense into 1 of volume 2746.0
Loaded syringe, remaining volume 0 uL
breaking dispense into 2 of volume 752.0
remaining vol:  949.0
remaining vol:  949.0
added water
breaking dispense into 1 of volume 733.0
remaining vol:  949.0
added ammonia
Loaded syringe, remaining volume 0 uL
breaking dispense into 3 of volume 900.0
Rinsing 1cc_4
Loaded syringe, remaining volume 0 uL
added 2700 f127
Loaded syringe, remaining volume 0 uL
breaking dispense into 3 of volume 730.0
Rinsing 1cc_4
Loaded syringe, remaining volume 0 uL
added 2190 uL CTAB
mixed samples
breaking dispense into 1 of volume 131.0
Time from start to TEOS addition: 834.8138771057129 s
########## Complete ############
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsamp

FileNotFoundError: [Errno 2] No such file or directory: './usaxs_queue/packet.txt'

In [109]:
usaxs_utils.trigger_usaxs_measurement(sample_name, sample_uid, 'ControlComposition')



trigger data:  ['2025-03-28 01:54:10', 'Control_3', 'ControlComposition', '3']
Error copying file: Command '['scp', '-i', '/Users/bgpelkie/silica-np-opt-key.pem', './usaxs_queue/packet.txt', 'ubuntu@18.216.44.137:~/usaxs_queue/']' returned non-zero exit status 255.


In [110]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

print(f'Time to load: {(time.time() - start_time)/60}')
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock, s_aspirate = 2000, s_dispense = 1000)

jubilee.park_tool()

end_time = time.time()

print(f'Total sample time: {(end_time - start_time)/60}')


Loaded syringe, remaining volume 0 uL
Time to load: 28.008575713634492
Total sample time: 29.92096585035324
